In [1]:
#import section 
import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans
import os

In [2]:
#importing and downloading mnist images from tensorflow 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
# one hot vectors is 0 in most dimension except for a single which 
# denotes a value.
full_data_x = mnist.train.images

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
%pwd

'C:\\Users\\Srinjay\\Desktop\\apollo_ai'

In [4]:
num_steps = 50 # Total epochs to train
batch_size = 1024 # The number of images per batch
k = 25 # The number of clusters
num_classes = 10 # number of clusters=np. of classes=10
num_features = 784 # Each image is 28x28 pixels(28*28=784)

In [5]:
# Input images
X = tf.placeholder(tf.float32, shape=[None, num_features])
# Labels (for assigning a label to a centroid and testing)
Y = tf.placeholder(tf.float32, shape=[None, num_classes])
#placeholder is not a variable. It is a value created for the tensorflow to run computation. 

In [6]:
#creating an object of KMeans class in tensoflow and apssing the arguments
#inputs=data to be clustered
#num_clusters=number of clusters data is divided into
#distance_metric=metric to be used to define the distance of a data point from the averagre or mean of the cluster
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine',
                use_mini_batch=True)

In [9]:
#creating instance of training_graph in kmeans
training_graph = kmeans.training_graph()
(all_scores, cluster_idx, scores, cluster_centers_initialized,
init_op, train_op) = training_graph

cluster_idx = cluster_idx[0] #cluster_idx being a tuple,taking the first value
#reduing the distance metric between the data points of the same cluster
avg_distance = tf.reduce_mean(scores)

# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()


(<tf.Tensor 'Squeeze_5:0' shape=<unknown> dtype=int64>,)


In [20]:
#Tensorflow uses a C++ backend. The connection to the backend is a session. Normally, a computation graph is 
# created and then the session is launched.
sess = tf.Session()
# Run the initializer
sess.run(init_vars, feed_dict={X: full_data_x})
sess.run(init_op, feed_dict={X: full_data_x})

# Training
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                         feed_dict={X: full_data_x})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))

# Assign a label to each centroid
# Count total number of labels per centroid, using the label of each training
# sample to their closest centroid (given by 'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
# Assign the most frequent label to the centroid
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

Step 1, Avg Distance: 0.341471
Step 10, Avg Distance: 0.221609
Step 20, Avg Distance: 0.220328
Step 30, Avg Distance: 0.219779
Step 40, Avg Distance: 0.219421
Step 50, Avg Distance: 0.219156


In [21]:
# Assign a label to each centroid
# Count total number of labels per centroid, using the label of each training
# sample to their closest centroid (given by 'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
# Assign the most frequent label to the centroid
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

In [22]:
# Evaluation ops
# Lookup: centroid_id -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
# Compute accuracy
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [23]:
#loading the validation data
test_x, test_y = mnist.test.images, mnist.test.labels
#printing the accuracy on validation data
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Test Accuracy: 0.7127
